In [74]:
!pip install transformers

NotImplementedError: ignored

In [70]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("question-answering", model="adityabhat/GPT3")

In [71]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering



In [72]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# tokenizer = AutoTokenizer.from_pretrained("hf-internal-testing/tiny-random-ReformerForQuestionAnswering")
# model = AutoModelForQuestionAnswering.from_pretrained("hf-internal-testing/tiny-random-ReformerForQuestionAnswering")

In [73]:
#!pip install transformers
!pip install evaluate

NotImplementedError: ignored

In [ ]:
# !pip install SentencePiece

In [ ]:
import pandas as pd

In [ ]:
df_qafinal = pd.read_csv('/content/drive/MyDrive/Booksummary/QA_Final.csv')

df_prompt_qa = pd.read_csv("/content/drive/MyDrive/Booksummary/Question_Prompts_with_QA.csv")

df_prompt_qa  = df_prompt_qa.loc[df_prompt_qa.QA.notna()][['Section_Id','Context_Id','Prompt']]
df_prompt_qa['Context'] = df_prompt_qa['Prompt'].str.replace('Text: ', "|").str.replace('Question: \n', "|").str.split("|").str[1].str.strip()

df_prompt_qa = df_prompt_qa[['Section_Id','Context_Id', 'Context']]

df_qafinal = df_qafinal.merge(df_prompt_qa, left_on="ContextId", right_on="Context_Id", how="left")
df_qafinal['Question'] = df_qafinal.Question.str.replace('Question: ','')
df_qafinal['Answer'] = df_qafinal.Answer.str.replace('Answer: ','')
data = df_qafinal[['QuesId','Question','Answer','Context']]
data.rename(columns={"Question":"question","Answer":"answer","Context":"context"},inplace=True)
data

In [ ]:
Question_Len = max(len(ques) for ques in data.question)
Answer_Len = max(len(ans) for ans in data.answer)
Question_Len,Answer_Len

In [ ]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]

        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)

        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100

        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [ ]:
import torch

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

In [ ]:
# TOKENIZER = T5TokenizerFast.from_pretrained("t5-base")
# MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# TOKENIZER = GPT2Tokenizer.from_pretrained("adityabhat/GPT3")
# MODEL = GPT2LMHeadModel.from_pretrained("adityabhat/GPT3")
MODEL_NAME = "gpt2"
MODEL = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
TOKENIZER = GPT2Tokenizer.from_pretrained(MODEL_NAME)

MODEL.to(DEVICE)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)

In [ ]:
Q_LEN = Question_Len # 256   # Question Length
T_LEN = Answer_Len #32  # Target Length
BATCH_SIZE = 2
# DEVICE = "cuda:0"

In [ ]:
# Dataloader

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [ ]:
import torch

train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(2):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["input_ids"].to(DEVICE)


        # Forward pass
        outputs = MODEL(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels

        )

        # Compute the loss
        loss = outputs.loss

        # Backpropagation and optimization
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()
        train_loss += loss.item()
        train_batch_count += 1

    # Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["input_ids"].to(DEVICE)


        # Forward pass
        with torch.no_grad():  # No need to compute gradients during evaluation
            outputs = MODEL(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

        # Compute the loss
        loss = outputs.loss
        val_loss += loss.item()
        val_batch_count += 1

    print(f"{epoch+1}/{2} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss / val_batch_count}")


In [ ]:
# # Load the GPT-2 model from the saved directory
# MODEL_PATH = "/content/drive/MyDrive/Booksummary/gpt2qa_model"
# MODEL = GPT2LMHeadModel.from_pretrained(MODEL_PATH)

# # Load the GPT-2 tokenizer from the saved directory
# TOKENIZER_PATH = "/content/drive/MyDrive/Booksummary/gpt2qa_tokenizer"
# TOKENIZER = GPT2Tokenizer.from_pretrained(TOKENIZER_PATH)


In [32]:
MODEL.save_pretrained("/content/drive/MyDrive/Booksummary/gptqa_model")
TOKENIZER.save_pretrained("/content/drive/MyDrive/Booksummary/gpt2qa_tokenizer")

# Saved files
"""('qa_tokenizer/tokenizer_config.json',
 'qa_tokenizer/special_tokens_map.json',
 'qa_tokenizer/spiece.model',
'qa_tokenizer/added_tokens.json',
'qa_tokenizer/tokenizer.json')"""

"('qa_tokenizer/tokenizer_config.json',\n 'qa_tokenizer/special_tokens_map.json',\n 'qa_tokenizer/spiece.model',\n'qa_tokenizer/added_tokens.json',\n'qa_tokenizer/tokenizer.json')"

In [65]:
def predict_answer(context, question, ref_answer=None):
    # Add the padding token to the tokenizer
    TOKENIZER.add_special_tokens({'pad_token': '[PAD]'})

    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_answer:
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer],
                            references=[ref_answer])

        print("Context: \n", context)
        print("\n")
        print("Question: \n", question)
        return {
            "Reference Answer: ": ref_answer,
            "Predicted Answer: ": predicted_answer,
            "BLEU Score: ": score
        }
    else:
        return predicted_answer


In [67]:
DEVICE

device(type='cuda')

In [66]:
i=10
predict_answer(data.iloc[i]['context'],data.iloc[i]['question'],data.iloc[i]['answer'],)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 144, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


RuntimeError: ignored

In [37]:
i=100
predict_answer(data.iloc[i]['context'],data.iloc[i]['question'],data.iloc[i]['answer'],)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 144, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Context: 
 "On the other hand, the Danavas, white as the clouds from which the rain hath dropped, possessing great strength and bold hearts, ascended the sky, and by hurling down thousands of mountains, continually harassed the gods. And those dreadful mountains, like masses of clouds, with their trees and flat tops, falling from the sky, collided with one another and produced a tremendous roar. And when thousands of warriors shouted without intermission in the field of battle and mountains with the woods thereon began to fall around, the earth with her forests trembled. Then the divine Nara appeared at the scene of the dreadful conflict between the Asuras and the Ganas (the followers of Rudra), and reducing to dust those rocks by means of his gold-headed arrows, he covered the heavens with dust. Thus discomfited by the gods, and seeing the furious discus scouring the fields of heaven like a blazing flame, the mighty Danavas entered the bowels of the earth, while others plunged into th

{'Reference Answer: ': 'The Danavas hurled down thousands of mountains from the sky to harass the gods according to the text.',
 'Predicted Answer: ': 'What did the Danavas hurl down from the sky to harass the gods according to the text?"On the other hand, the Danavas, white as the clouds from which the rain hath dropped, possessing great strength and bold hearts, ascended the sky, and by hurling down thousands of mountains, continually harassed the gods. And those dreadful mountains, like masses of clouds, with their trees and flat tops, falling from the sky, collided with one another and produced a tremendous roar. And when thousands of warriors shouted without intermission in the field of battle and mountains with the woods thereon began to fall around, the earth with her forests trembled. Then the divine Nara appeared at the scene of the',
 'BLEU Score: ': {'google_bleu': 0.09363295880149813}}

In [51]:
i=100
pred_ans = predict_answer(data.iloc[i]['context'],data.iloc[i]['question'],data.iloc[i]['answer'],)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 144, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Context: 
 "On the other hand, the Danavas, white as the clouds from which the rain hath dropped, possessing great strength and bold hearts, ascended the sky, and by hurling down thousands of mountains, continually harassed the gods. And those dreadful mountains, like masses of clouds, with their trees and flat tops, falling from the sky, collided with one another and produced a tremendous roar. And when thousands of warriors shouted without intermission in the field of battle and mountains with the woods thereon began to fall around, the earth with her forests trembled. Then the divine Nara appeared at the scene of the dreadful conflict between the Asuras and the Ganas (the followers of Rudra), and reducing to dust those rocks by means of his gold-headed arrows, he covered the heavens with dust. Thus discomfited by the gods, and seeing the furious discus scouring the fields of heaven like a blazing flame, the mighty Danavas entered the bowels of the earth, while others plunged into th

In [ ]:
pred_answers=[]
bleu_score=[]
for i in range(len(data)):
  pred_ans = predict_answer(data.iloc[i]['context'],data.iloc[i]['question'],data.iloc[i]['answer'],)
  pred_answers.append(pred_ans['Predicted Answer: '])
  bleu_score.append(pred_ans['BLEU Score: '])


In [ ]:
predict_answer(data.iloc[i]['context'],data.iloc[i]['question'],data.iloc[i]['answer'],)

In [ ]:
df1 = pd.DataFrame({"pred_answer": pred_answers, "bleu_score":bleu_score})
df1.index = data.index

val_data_index = val_data.index

df1['train_data']=False

for i in val_data_index:
  df1.loc[i,"train_data"]=True

In [ ]:
df1.to_csv('/content/drive/MyDrive/Booksummary/t5predicted_ans.csv')

In [ ]:
df1

,pred_answer,bleu_score,train_data
0,"The Mahabharata was composed by Ugrasrava, the...",{'google_bleu': 0.21621621621621623},False
1,"The Mahabharata is a sacred and sacred book, w...",{'google_bleu': 0.09340659340659341},False
2,The creation of the world began according to t...,{'google_bleu': 0.02608695652173913},False
3,The universe will be affected at the end of th...,{'google_bleu': 0.19801980198019803},True
4,The generation of Devas was thirty-three thous...,{'google_bleu': 0.06204379562043796},False
...,...,...,...
1114,Agni responds to Drona's request by saying tha...,{'google_bleu': 0.16216216216216217},False
1115,"Jarita refers to the ""eldest"" and ""next"" among...",{'google_bleu': 0.1111111111111111},False
1116,Mandapala says that co-wives and clandestine l...,{'google_bleu': 0.14},True
1117,Mandapala compares Jarita's jealousy to the je...,{'google_bleu': 0.14912280701754385},False
